In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_classification

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
%matplotlib inline
from matplotlib import pyplot as plt
from flask import json, jsonify

In [2]:
employment_county_df  = pd.read_csv('../data_files/Employment_by_County.csv', index_col=0)
employment_county_df.head(5)

,county,labor_force,employed,unemployed,unemployed_pct,population,race_white,race_black,race_native,race_asian,race_islander,race_other,race_two_or_more
geoid,,,,,,,,,,,,,
1001,"Autauga County, Alabama",25838,24576,1262,4.9,58805,42160,11445,217,881,35,910,3157
1003,"Baldwin County, Alabama",96763,91338,5425,5.6,231767,189399,18217,1582,2067,143,5335,15024
1005,"Barbour County, Alabama",8587,7982,605,7.0,25223,11317,11933,116,117,1,1039,700
1007,"Bibb County, Alabama",8640,8067,573,6.6,22293,16555,4413,60,32,9,465,759
1009,"Blount County, Alabama",24661,23653,1008,4.1,59134,50663,845,337,178,24,3431,3656


In [3]:
employment_county_copy = employment_county_df.copy()

In [4]:
employment_county_copy['race_others']= employment_county_copy[['race_native','race_islander','race_other']].sum(axis=1)
employment_county_copy.head(5)

,county,labor_force,employed,unemployed,unemployed_pct,population,race_white,race_black,race_native,race_asian,race_islander,race_other,race_two_or_more,race_others
geoid,,,,,,,,,,,,,,
1001,"Autauga County, Alabama",25838,24576,1262,4.9,58805,42160,11445,217,881,35,910,3157,1162
1003,"Baldwin County, Alabama",96763,91338,5425,5.6,231767,189399,18217,1582,2067,143,5335,15024,7060
1005,"Barbour County, Alabama",8587,7982,605,7.0,25223,11317,11933,116,117,1,1039,700,1156
1007,"Bibb County, Alabama",8640,8067,573,6.6,22293,16555,4413,60,32,9,465,759,534
1009,"Blount County, Alabama",24661,23653,1008,4.1,59134,50663,845,337,178,24,3431,3656,3792


In [5]:
# reducing columns to improve model performance 
employment_county_copy = employment_county_copy.drop(['race_native','race_islander','race_other'],axis=1)

In [6]:
employment_county_copy.isnull().values.any()

False

In [7]:
employment_county_copy.shape

(3125, 11)

In [8]:
employment_county_copy.columns.to_list()

['county',
 'labor_force',
 'employed',
 'unemployed',
 'unemployed_pct',
 'population',
 'race_white',
 'race_black',
 'race_asian',
 'race_two_or_more',
 'race_others']

In [9]:
employment_county_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3125 entries, 1001 to 56045
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   county            3125 non-null   object 
 1   labor_force       3125 non-null   int64  
 2   employed          3125 non-null   int64  
 3   unemployed        3125 non-null   int64  
 4   unemployed_pct    3125 non-null   float64
 5   population        3125 non-null   int64  
 6   race_white        3125 non-null   int64  
 7   race_black        3125 non-null   int64  
 8   race_asian        3125 non-null   int64  
 9   race_two_or_more  3125 non-null   int64  
 10  race_others       3125 non-null   int64  
dtypes: float64(1), int64(9), object(1)
memory usage: 293.0+ KB


In [10]:
X = employment_county_copy[['race_white', 'race_black', 'race_asian', 'race_two_or_more', 'race_others']]
y = employment_county_copy['unemployed_pct'] 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, random_state=0)

In [12]:
model = RandomForestRegressor(n_estimators=10,random_state=0)
model.fit(X_train,y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

In [13]:
y_pred = model.predict(X_test)

In [14]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.23167471422209096

In [15]:
model.score(X_train,y_train)

0.868011884600839

In [16]:
model.score(X_test,y_test)

0.23167471422209096

In [17]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)
mae

1.4762400000000002

In [25]:
X.columns

Index(['race_white', 'race_black', 'race_asian', 'race_two_or_more',
       'race_others'],
      dtype='object')

## Feature importance

In [23]:
feature_importance = sorted(zip(model.feature_importances_, X.columns), reverse=True)
#feature_importance = json.dumps(feature_importance)

In [24]:
feature_importance

[(0.23025521088268416, 'race_black'),
 (0.22794818056346577, 'race_others'),
 (0.20405346968459162, 'race_white'),
 (0.19331122812071583, 'race_two_or_more'),
 (0.1444319107485426, 'race_asian')]

### MAE seem a bit high, but consider the min max range

In [21]:
max_labor_force = employment_county_df['unemployed_pct'].max()
min_labor_force = employment_county_df['unemployed_pct'].min()
print ('min = ', min_labor_force)
print('max = ', max_labor_force)

min =  1.7
max =  22.5


In [22]:
print(f'Actual:\t\t{list(y_test[:10])}')
print(f'Predicted:\t{list(model.predict(X_test[:10]))}')

Actual:		[7.4, 4.2, 7.6, 5.3, 7.0, 4.9, 6.7, 8.3, 3.3, 6.2]
Predicted:	[7.08, 3.6900000000000004, 8.030000000000001, 6.05, 6.1, 6.1499999999999995, 6.58, 7.110000000000001, 7.51, 6.65]


In [26]:
filename = '../Flask_presentation/static/model/finalized_model_unemployment_pct.sav'
pickle.dump(model, open(filename, 'wb'))